In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")

print(USERNAME)

neo4j


In [3]:
import pandas as pd
df = pd.read_csv("cleaned_medicine_data.csv")

df.sample(4)

,medicine_name,composition,uses,side_effects,manufacturer,chemical_class,habit_forming,therapeutic_class,action_class,substitutes
10686,volibo 0 3 tablet,Voglibose (0.3mg),Treatment of Type 2 diabetes mellitus,Skin rash Flatulence Abdominal pain Diarrhea,Sun Pharmaceutical Industries Ltd,Valiolamine derivative,No,ANTI DIABETIC,Alpha-glucosidase inhibitors,"Vozuca Activ 0.3 Tablet, Voglit 0.3mg Tablet, ..."
6630,olmetor 20 tablet,Olmesartan Medoxomil (20mg),Hypertension (high blood pressure) Heart fail...,Cough Diarrhea Dizziness Flu like symptoms Hea...,Torrent Pharmaceuticals Ltd,Synthetic imidazole derivative,No,CARDIAC,Angiotensin receptor blockers(ARB),"Olmighty 20 Tablet, Olmetime 20 Tablet, Olsert..."
2719,genticyn 80mg injection,Gentamicin (80mg),Treatment of Bacterial infections,Hearing loss Kidney damage Balance disorder lo...,Abbott,Aminoglycosides,No,OPHTHAL OTOLOGICALS,Aminoglycosides,"Germenta 80mg Injection, Gmi 80mg Injection, G..."
1784,enam 5 tablet,Enalapril (5mg),Treatment of Hypertension (high blood pressure...,Decreased blood pressure Cough Increased potas...,Dr Reddy's Laboratories Ltd,Dipeptides,No,CARDIAC,Angiotensin-converting enzyme (ACE) inhibitors,"Enpril 5mg Tablet, Converten 5mg Tablet, Fribi..."


In [4]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(USERNAME, PASSWORD))

In [5]:
# Function to insert medicine data into the graph
def create_medicine_graph(tx, row):
    # Handle NaN values by replacing them with a placeholder or skipping them
    medicine_name = row["medicine_name"] if pd.notna(row["medicine_name"]) else "Unknown"
    composition = row["composition"] if pd.notna(row["composition"]) else "Unknown"
    uses = row["uses"] if pd.notna(row["uses"]) else "Unknown"
    side_effects = row["side_effects"] if pd.notna(row["side_effects"]) else "Unknown"
    manufacturer = row["manufacturer"] if pd.notna(row["manufacturer"]) else "Unknown"
    chemical_class = row["chemical_class"] if pd.notna(row["chemical_class"]) else "Unknown"
    therapeutic_class = row["therapeutic_class"] if pd.notna(row["therapeutic_class"]) else "Unknown"
    action_class = row["action_class"] if pd.notna(row["action_class"]) else "Unknown"
    substitutes = row["substitutes"] if pd.notna(row["substitutes"]) else "Unknown"

    # Cypher query to insert or merge nodes and relationships
    query = """
    MERGE (m:Medicine {name: $medicine_name})
    SET m.composition = $composition
    MERGE (u:UseCase {name: $uses})
    MERGE (s:SideEffect {name: $side_effects})
    MERGE (mf:Manufacturer {name: $manufacturer})
    MERGE (cc:ChemicalClass {name: $chemical_class})
    MERGE (tc:TherapeuticClass {name: $therapeutic_class})
    MERGE (ac:ActionClass {name: $action_class})
    MERGE (sub:Substitute {name: $substitutes})
    
    MERGE (m)-[:TREATS]->(u)
    MERGE (m)-[:HAS_SIDE_EFFECT]->(s)
    MERGE (m)-[:MANUFACTURED_BY]->(mf)
    MERGE (m)-[:BELONGS_TO]->(cc)
    MERGE (m)-[:HAS_THERAPEUTIC_CLASS]->(tc)
    MERGE (m)-[:HAS_ACTION_CLASS]->(ac)
    MERGE (m)-[:HAS_SUBSTITUTE]->(sub)
    """
    
    # Run the query with cleaned data
    tx.run(query, 
           medicine_name=medicine_name, 
           composition=composition, 
           uses=uses, 
           side_effects=side_effects, 
           manufacturer=manufacturer, 
           chemical_class=chemical_class, 
           therapeutic_class=therapeutic_class, 
           action_class=action_class, 
           substitutes=substitutes)


In [6]:
with driver.session() as session:
    for _, row in df.iterrows():
        session.write_transaction(create_medicine_graph, row)

print("Drug Knowledge Graph successfully created in Neo4j!")

C:\Users\aksha\AppData\Local\Temp\ipykernel_21872\3413950902.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_medicine_graph, row)


Drug Knowledge Graph successfully created in Neo4j!
